In [1]:
import pandas as pd

# ---------------- Pandas settings --------------- #
# Removes rows and columns truncation of '...'
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

## Analyzing the Saltiness of the Users

In [2]:
## Completely Optional Code below, only try if running Jupyter Lab doesn't find postgres_helper.py
import sys, os
cwd = os.getcwd()
sys.path[0] = cwd[:cwd.rfind('/')]

## Create Connection to PostgreSQL db

In [3]:
from data import postgres_helper as pgh

pg_conn = pgh.create_connection(verbose=True)

Environment file loaded successfully...
Using psycopg2 version: 2.8.4 (dt dec pq3 ext lo64)
Creating Connection to txtqhcho, txtqhcho@salt.db.elephantsql.com...
PostgreSQL database version: ('PostgreSQL 11.5 (Ubuntu 11.5-1.pgdg18.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.4.0-1ubuntu1~18.04.1) 7.4.0, 64-bit',)


## List all tables in the database

In [4]:
pgh.get_sql_tables(pg_conn)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,saltyhackers,txtqhcho,None,False,False,False,False


## Schema of `saltyhackers` table

In [5]:
table_info_query = """SELECT column_name, udt_name, data_type, character_maximum_length
FROM information_schema.columns
WHERE table_name   = 'saltyhackers'
;
"""
pgh.select_query(pg_conn, table_info_query)

,column_name,udt_name,data_type,character_maximum_length
0,id,int8,bigint,None
1,by,text,text,None
2,author,text,text,None
3,time,int8,bigint,None
4,time_ts,timestamptz,timestamp with time zone,None
5,text,text,text,None
6,parent,int8,bigint,None
7,deleted,text,text,None
8,dead,text,text,None
9,ranking,int8,bigint,None


## List Top 10 rows from the `saltyhackers` table

In [6]:
top_10_query = """SELECT * FROM saltyhackers LIMIT 10;"""
pgh.select_query(pg_conn, top_10_query)

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking,score
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0,-0.0616
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0,0.0000
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0,0.5215
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0,1.0102
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0,0.0000
5,3867418,Iv,Iv,1334921984,2012-04-20 11:39:44+00:00,> There's a whole class of 'mom & pop' type we...,3867404,None,None,0,-0.4432
6,3925617,Iv,Iv,1336076765,2012-05-03 20:26:05+00:00,I'm also in this ballpark (300-600€ / day) whi...,3924840,None,None,0,0.0000
7,3107534,Iv,Iv,1318520044,2011-10-13 15:34:04+00:00,how do you run unity in non-accelerated mode ?...,3107241,None,None,0,-0.3950
8,8409259,Iv,Iv,1412421647,2014-10-04 11:20:47+00:00,Polio is not exterminated even if it is absent...,8409226,None,None,0,-0.2633
9,2855741,Jd,Jd,1312690646,2011-08-07 04:17:26+00:00,"Yep, I didn't find Rescuetime very helpful. I ...",2855343,None,None,0,-0.5840


### Find Top 100 Salty Users

In [10]:
query = """SELECT parent, id, time, by, text, score FROM saltyhackers
WHERE score < 0
ORDER BY score ASC
LIMIT 100;"""
pgh.select_query(pg_conn, query)

,parent,id,time,by,text,score
0,6972199,6972554,1388177319,aet,"Fuck war, too? Right?",-1.6249
1,7426842,7432665,1395272201,koberstein,Fear of death is the actual problem.,-1.5789
2,4553433,4554287,1348243431,fsckin,Would you steal from a terrorist?,-1.5610
3,1257893,1258014,1271031774,Sandman,"Unfortunately, no.",-1.5574
4,5144557,5144668,1359636143,chris_wot,So he can't learn from a competitor. Blackberr...,-1.5483
5,9219519,9219576,1426612986,fweespeech,"No worries, written communication isn't perfect.",-1.5175
6,4340811,4341002,1344158051,antihero,"As opposed to, for instance, rapists, war crim...",-1.5149
7,8472268,8472948,1413575233,MadManE,Bad decisions make people poor. Being poor doe...,-1.4907
8,7280275,7280700,1393035324,nwh,"Ugh, that's unbelievably awful.",-1.4794
9,9255943,9259233,1427228828,hobarrera,No development + no source + time = broken.,-1.4719
